In [1]:
# Do all imports here

import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import pandas as pd



### Get all pages and books since February 11, 2011

##### Get all Sundays since February 13, 2011 in ```YYYY/MM/DD``` format

In [2]:

# Define the start and end dates
start_date = datetime(2011, 2, 13)
end_date = datetime.now()

# Calculate the number of days between start and end dates
delta = end_date - start_date

# Iterate over the days and print Sundays in YYYY/MM/DD format
all_sundays = []
for i in range(delta.days + 1):
    current_date = start_date + timedelta(days=i)
    if current_date.weekday() == 6:  # Sunday has index 6
        sunday = current_date.strftime("%Y/%m/%d")
        all_sundays.append(sunday)


##### Now create a loop to change url with each date and scrape the data and add a key to the dictionary with the date

In [4]:
all_pages = []
for date in all_sundays :

# change url here 

    response = requests.get(f"https://www.nytimes.com/books/best-sellers/{date}/combined-print-and-e-book-fiction/")
    soup_doc = BeautifulSoup(response.text)
    # get a list of dictionary with info for all books

    tag_names = soup_doc.select('.css-13y32ub')
    
    all_books = []

    rank = 0

    for tag_name in tag_names :
        rank = rank+ 1
        week = tag_name.select_one(".css-1o26r9v").text
        name = tag_name.select_one(".css-5pe77f").text
        author = tag_name.select_one(".css-hjukut").text.split("by ")[1]
        publisher = tag_name.select_one(".css-heg334").text
        description = tag_name.select_one(".css-14lubdp").text
        one_page = {
            'rank' : rank,
            'no_of_weeks' : week,
            'book' : name,
            'author' : author,
            'publisher' : publisher,
            'description' : description,
            'date' : date
        }
        all_books.append(one_page)
    all_pages.extend(all_books)
    


##### Add all this to a pandas dataframe

In [5]:
df = pd.DataFrame(all_pages)
df

,rank,no_of_weeks,book,author,publisher,description,date
0,1,New this week,TICK TOCK,James Patterson and Michael Ledwidge,"Little, Brown",The New York detective Michael Bennett enlists...,2011/02/13
1,2,New this week,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,Knopf Doubleday,A hacker and a journalist investigate the disa...,2011/02/13
2,3,New this week,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,Knopf Doubleday,In the second volume of the Millennium trilogy...,2011/02/13
3,4,New this week,THE GIRL WHO KICKED THE HORNET’S NEST,Stieg Larsson,Knopf Doubleday,"The third volume of the Millennium trilogy, ab...",2011/02/13
4,5,New this week,WATER FOR ELEPHANTS,Sara Gruen,Algonquin,"After his parents die in a car accident, young...",2011/02/13
...,...,...,...,...,...,...,...
9985,11,54 weeks on the list,LESSONS IN CHEMISTRY,Bonnie Garmus,Doubleday,A scientist and single mother living in Califo...,2023/12/03
9986,12,4 weeks on the list,THE SECRET,Lee Child and Andrew Child,Delacorte,The 28th book in the Jack Reacher series. It’s...,2023/12/03
9987,13,3 weeks on the list,DIRTY THIRTY,Janet Evanovich,Atria,The 30th book in the Stephanie Plum series. Pl...,2023/12/03
9988,14,22 weeks on the list,ICEBREAKER,Hannah Grace,Atria,Anastasia might need the help of the captain o...,2023/12/03


##### Splitting authors into two where there is more than one

In [6]:
df[['author1', 'author2']] = df['author'].str.split(' and ', expand=True)
df.to_csv('nyt_raw.csv', index=False)

In [132]:
# # get a separate dataframe for unique first authors and export to csv

# unique_author_1 = df['author1'].unique()

# df_authors_1 = pd.DataFrame({'author1': unique_author_1})

# df_authors_1.to_csv('authors_1.csv')

In [143]:
# # also get  aunique list of books 

# unique_books = df['book'].unique()
# df_books = pd.DataFrame({'book_name': unique_books})
# df_books.to_csv('book_list.csv')